
# **Giotto-TDA Challenge**
# Predicting Volcano Eruption
   

**Summary:**In this approach we show how TDA can be an effective tool for regression on **multivariate** time series analysis. We compare the TDA approach with a standard baseline approach and after show that a respectable performance can be achieved by merging both. We validate our results by comparing with the Kaggle leaderboard.

**Data:** We'll be with the sensor data of several volcanoes, the objective is to predict the time untill eruption based on each volcano's sensor data. For runtime reasons we will be working with only 1/9 of the data (the full dataset is 20Gb). The full data along with the competition can be found here: https://www.kaggle.com/c/predict-volcanic-eruptions-ingv-oe/overview



# **I.**  The Task

Earthquakes are devastating fenomena whose damage is not only human but material. The most challenging aspects of sismic behaviour is its unpredictability. But what is it was possible to predict earthquakes in advance as such as other environmental occurences such as the weather? Current estimates are only reliable a couple of minutes in advance and they usually fail at longer-term predictions.

Italy's Istituto Nazionale di Geofisica e Vulcanologia (INGV), with its focus on geophysics and volcanology, has issued a challenge regarding this task. (https://www.kaggle.com/c/predict-volcanic-eruptions-ingv-oe/overview).


**Task Description:** To predict "time to eruption” by surveying volcanic tremors from seismic signals. 

**Data Description:** The data has 1000 volcanoes, each volcano has 10 sensors. Each sensor is a time-series data. The objective is to predict the time it will take for the volcano to erupt given the data in each sensor.

<img src="images/giotto_first_fig.png">

Fig.1 Illustration of a single observation of the dataset. For each volcano there are 10 sensors, each sensor is a time-series.

# **II.** Libraries 

We will use tensorflow and xgboost for regression (aside from the regular libraries such as pandas, numpy and giotto-tda)

In [ ]:
import sys
!{sys.executable} -m pip install xgboost tensorflow sklearn tensorflow-addons

In [1]:
#Data wrangling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

#TDA
from gtda.diagrams import PersistenceEntropy, Scaler, PairwiseDistance, Amplitude
from gtda.homology import VietorisRipsPersistence
from gtda.metaestimators import CollectionTransformer
from gtda.pipeline import Pipeline
from gtda.time_series import TakensEmbedding, PearsonDissimilarity
from gtda.plotting import plot_diagram


#Benchmarking
import tensorflow as tf
import tensorflow.keras.backend as bk
import tensorflow.keras.layers as ly
import tensorflow.keras.models as ml
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
import tensorflow_addons as tfa
import xgboost
from sklearn.model_selection import KFold

# **III.** The Data

This is just 1/8 of the whole dataset. We download it directly from kaggle but for that we need its API

In [2]:
!{sys.executable} -m pip install kaggle
!mkdir ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Defaulting to user installation because normal site-packages is not writeable
  Using cached kaggle-1.5.9.tar.gz (58 kB)
  Using cached tqdm-4.51.0-py2.py3-none-any.whl (70 kB)
  Using cached python-slugify-4.0.1.tar.gz (11 kB)
  Using cached slugify-0.0.1.tar.gz (1.2 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Created wheel for kaggle: filename=kaggle-1.5.9-py3-none-any.whl size=73265 sha256=42aa4ee64515118260e2a7581ecfeaa90dd3422ff1fd65864a86eef20e43fdac
  Stored in directory: /home/antonio/.cache/pip/wheels/9d/50/3d/2644504bb1e8c782f3fef5984f03d76fc4a74698fdec128b29
  Created wheel for python-slugify: filename=python_slugify-4.0.1-py2.py3-none-any.whl size=6767 sha256=e3e138a8db0cb53d99bde1db4dbe917f3481614a95a9c30ee8a0eb4d8147e6eb
  Stored in directory: /home/antonio/.cache/pip/wheels/72/e6/db/122611605e60148f54ee2abaca98b2bbeafc6e22486a867bad
  Created wheel for slugify: filename=slugify-0.0.1-py3-none-any.whl size=1908 sha256=9efbb25090dbe07d5f46c03c90e95

In [4]:
#Download the data
!kaggle datasets download -d antnioleito/volcano-data

100%|█████████████████████████████████████▉| 1.15G/1.15G [02:31<00:00, 8.41MB/s]
100%|██████████████████████████████████████| 1.15G/1.15G [02:31<00:00, 8.16MB/s]


In [7]:
#unzip the data
import zipfile
with zipfile.ZipFile('volcano-data.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [2]:
volcano_ids = [item for item in os.listdir('data/')]
labels = pd.read_csv('train.csv')


<span style="color:#2F4F4F">**volcano_ids**</span> has the id of each volcano, each volcano has a dataframe of 60000 entries for each of its 10 sensors

In [3]:
temp = pd.read_csv('data/'+volcano_ids[0])
temp.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
0,-534.0,-1217.0,-151.0,244.0,412.0,438.0,67.0,60.0,-31.0,15.0
1,-364.0,-1160.0,-222.0,339.0,318.0,64.0,143.0,323.0,-137.0,111.0
2,-71.0,-1068.0,-228.0,514.0,315.0,-152.0,184.0,-96.0,-246.0,-54.0
3,-113.0,-1059.0,-259.0,627.0,324.0,-188.0,196.0,-22.0,-335.0,-377.0
4,-106.0,-1038.0,-311.0,760.0,267.0,-275.0,234.0,-188.0,-417.0,-629.0


<span style="color:#2F4F4F">**labels**</span> has the id of each volcano and the "time to erupt" our regression target. 

In [4]:
labels.head()

,segment_id,time_to_eruption
0,1136037770,12262005
1,1969647810,32739612
2,1895879680,14965999
3,2068207140,26469720
4,192955606,31072429


Lastly we define a preprocessing function that fills the nan values with a window average.

In [5]:
def preprocess(volcano_name):
    series = pd.read_csv('data/'+volcano_name)
    return series.fillna(series.rolling(10,min_periods=1).mean()).dropna(axis=0)

# **IV.** A Topological Approach

Our approach is to do a Taken's Embedding for each one of the sensors. After, we compute the Vietoris-Tips filtration and extract persistence summaries: Entropy and Amplitude

### **IV.a Takens' embedding on multivariate time series**


<img src="images/giotto_sec_fig.png">

In [6]:
#Creating a The pipeline
embedding_dimension = 10
embedding_time_delay = 1
stride = 100

#Takens Embedding
embedder = TakensEmbedding(time_delay=embedding_time_delay,
                           dimension=embedding_dimension,
                           stride=stride)
#Persistent Homology
persistence = VietorisRipsPersistence(homology_dimensions=[0, 1], n_jobs=-1)
steps = [("embedder", embedder),
         ("persistence", persistence)]
transfomer = Pipeline(steps)

#After calculate also the amplitude and Entropy
amp = Amplitude()
ent = PersistenceEntropy()

In [10]:
#Apply the takens embedding and get the persistence diagrams for every volcano (diags)
#This is the most time-consuming part which should take around 10 minutes.

diags=[]
for volcano in volcano_ids[:-1]:
    volcano_data = preprocess(volcano)
    diags.append(transfomer.fit_transform(volcano_data.values.T) )

#Get the H0 and H1 entropy and amplitude of each peristince diagram of each volcano   
amplitude=[]
entropy=[]
ws = []
for diag in diags:
    amplitude.append(amp.fit_transform(diag))
    entropy.append(ent.fit_transform(diag))

/home/antonio/.local/lib/python3.6/site-packages/gtda/homology/simplicial.py:209: DataDimensionalityWarning:

Input array X has X.shape[1] == X.shape[2]. This is consistent with a collection of distance/adjacency matrices, but the input is being treated as a collection of vectors in Euclidean space.

/home/antonio/.local/lib/python3.6/site-packages/gtda/homology/simplicial.py:271: DataDimensionalityWarning:

Input array X has X.shape[1] == X.shape[2]. This is consistent with a collection of distance/adjacency matrices, but the input is being treated as a collection of vectors in Euclidean space.

/home/antonio/.local/lib/python3.6/site-packages/gtda/homology/simplicial.py:209: DataDimensionalityWarning:

Input array X has X.shape[1] == X.shape[2]. This is consistent with a collection of distance/adjacency matrices, but the input is being treated as a collection of vectors in Euclidean space.

/home/antonio/.local/lib/python3.6/site-packages/gtda/homology/simplicial.py:271: DataDimensio

In [21]:
import gtda

In [22]:
print(gtda.__version__)

0.3.0


### **IV.b Build new features.**


<img src="images/giotto_trd_fig.png">

We now build a new dataset. For each observation (volcano) we have the topological variables just calculated. Lastly we bring the "time_to_erupt" variable, our regression target.

In [20]:
#merge H0 and H1 entropy and amplitude into columns
X_topo = np.c_[np.array(amplitude)[:,:,0],np.array(amplitude)[:,:,1],
               np.array(entropy)[:,:,0],np.array(entropy)[:,:,1]]

#Grab the regression target
y=np.array([labels[labels['segment_id']==np.int(name[:-4])]['time_to_eruption'].values[0] for name in volcano_ids[:-1]])

# **V.** Baseline

In [13]:
def createANN(X):
    model = ml.Sequential()
    model.add(ly.Input(X.shape[1]))
    model.add(ly.BatchNormalization())
    model.add(tfa.layers.WeightNormalization(ly.Dense(1000,activation='relu')))
    model.add(ly.BatchNormalization())
    model.add(ly.Dropout(0.7))
    model.add(tfa.layers.WeightNormalization(ly.Dense(1,activation='relu')))


    model.compile(optimizer=tfa.optimizers.AdamW(lr = 1, weight_decay = 1e-5, clipvalue = 900),loss='mean_absolute_error')
    return model



In [25]:
def benchmark(X,y, k=10):
    kf = KFold(n_splits=k)
    kf.get_n_splits(X)
    
    ann_scores =[]
    xgb_scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]
        
        #ANN
        cb_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_lr=1e-7, patience=2, verbose=0, mode='min')
        cb_early = EarlyStopping(monitor="val_loss", mode="min", restore_best_weights=True, patience= 5, verbose = 0)
        model=createANN(X)
        model.fit(X_train,y_train,batch_size=8,epochs=600,verbose=0,validation_data=(X_val,y_val),callbacks=[cb_lr,cb_early])

        #XGBOOST
        model1 = xgboost.XGBRegressor(n_estimators=100000,max_depth=8,learning_rate=0.05,alpha=0.1,SUBSAMPLE=0.6)
        eval_set = [(X_val, y_val)]
        model1.fit(X_train, y_train,early_stopping_rounds=5,eval_metric='mae', eval_set=eval_set, verbose=False)
        
        ann_scores.append(np.mean(np.abs(model.predict(X_val)-y_val)))
        xgb_scores.append(np.mean(np.abs(model1.predict(X_val)-y_val)))
    
    return np.mean(ann_scores), np.mean(xgb_scores)
        
        

In [26]:
ann, xgb = benchmark(X_topo, y, k=10)

[02:11:57] WARNING: ../src/learner.cc:516: 
Parameters: { SUBSAMPLE } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:12:01] WARNING: ../src/learner.cc:516: 
Parameters: { SUBSAMPLE } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:12:04] WARNING: ../src/learner.cc:516: 
Parameters: { SUBSAMPLE } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:12:07] WA

In [27]:
print(f'Neural Network 10-fold MAE: {ann}')
print(f'XGBoost 10-fold MAE: {xgb}')

Neural Network 10-fold MAE: 11958279.017435
XGBoost 10-fold MAE: 6166860.9578125


# **VI.** Comparison

Here we compare out method with the baseline one. This approach was taked from the most voted baseline notebook at this kaggle competition: https://www.kaggle.com/soheild91/ingv-nn-xgb-baseline

The idea is similar but the features extracted from each sensor are not the topological ones but simple statistical ones. While we took one 2 features per sensor, the standard baseline takes 12 new features per sensor

In [28]:
new_features=12
base_data=np.empty((len(volcano_ids[:-1]),new_features*10))
for i_ in range(len(volcano_ids[:-1])):
    the_df=preprocess(volcano_ids[i_])
    base_data[i_,:]=np.concatenate((the_df.abs().mean().to_numpy(),
                                    the_df.std().to_numpy(),
                                    the_df.mean().to_numpy(),
                                    the_df.var().to_numpy(),
                                    the_df.min().to_numpy(),
                                    the_df.max().to_numpy(),
                                    the_df.median().to_numpy(),
                                    the_df.quantile([0.1,0.25,0.5,0.75,0.9]).to_numpy().reshape(1,-1)[0]))

In [29]:
stats = benchmark(base_data,y,k=10)
together = benchmark(np.c_[X_topo,base_data],y,k=10)

[02:14:27] WARNING: ../src/learner.cc:516: 
Parameters: { SUBSAMPLE } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:14:32] WARNING: ../src/learner.cc:516: 
Parameters: { SUBSAMPLE } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:14:36] WARNING: ../src/learner.cc:516: 
Parameters: { SUBSAMPLE } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:14:41] WA

# **VII.** Conclusion

In [31]:
print('Just Topological features')
print(f'Neural Network 10-fold MAE: {ann}')
print(f'XGBoost 10-fold MAE: {xgb}')
print('\n')
print('Just Simple Statistics')
print(f'Neural Network 10-fold MAE: {stats[0]}')
print(f'XGBoost 10-fold MAE: {stats[1]}')
print('\n')
print('Both together')
print(f'Neural Network 10-fold MAE: {together[0]}')
print(f'XGBoost 10-fold MAE: {together[1]}')

Just Topological features
Neural Network 10-fold MAE: 11958279.017435
XGBoost 10-fold MAE: 6166860.9578125


Just Simple Statistics
Neural Network 10-fold MAE: 12119103.21565
XGBoost 10-fold MAE: 5642745.1707500005


Both together
Neural Network 10-fold MAE: 12108888.428183125
XGBoost 10-fold MAE: 5364914.2945


### Note that we are only using 1/8 of the whole dataset we still manage to get a very competitive approach. Note that the simple statistical approach has 6 times more features. The results are nothing short of impressive specially when considering both approaches combined.

Below is the kaggle leaderboard for comparison.

<img src="images/giotto_leaderboard.png">

## Authors: António Leitão and Giovanni Petri